In [1]:
import pandas as pd
train=pd.read_csv('../input/water-potability-prediction/train_dataset.csv')
train.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.080795,219.674262,22210.613083,5.875041,333.775777,398.517703,11.502316,112.412210,2.994259,0
1,6.783888,193.653581,13677.106441,5.171454,323.728663,477.854687,15.056064,66.396293,3.250022,0
2,6.010618,184.558582,15940.573271,8.165222,421.486089,314.529813,20.314617,83.707938,4.867287,1
3,8.097454,218.992436,18112.284447,6.196947,333.775777,376.569803,17.746264,59.909941,4.279082,1
4,8.072612,210.269780,16843.363927,8.793459,359.516169,559.167574,17.263576,68.738989,5.082206,0


In [2]:
train.nunique()

ph                 1948
Hardness           2293
Solids             2293
Chloramines        2293
Sulfate            1766
Conductivity       2293
Organic_carbon     2293
Trihalomethanes    2193
Turbidity          2293
Potability            2
dtype: int64

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293 entries, 0 to 2292
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2293 non-null   float64
 1   Hardness         2293 non-null   float64
 2   Solids           2293 non-null   float64
 3   Chloramines      2293 non-null   float64
 4   Sulfate          2293 non-null   float64
 5   Conductivity     2293 non-null   float64
 6   Organic_carbon   2293 non-null   float64
 7   Trihalomethanes  2293 non-null   float64
 8   Turbidity        2293 non-null   float64
 9   Potability       2293 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 179.3 KB


In [4]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [5]:
test=pd.read_csv('../input/water-potability-prediction/test_dataset.csv')
test.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,8.813505,236.392817,40684.390014,6.608775,303.529818,278.355123,14.576057,72.872697,4.242460
1,8.466013,224.174936,22523.124552,6.668868,286.943922,330.044715,17.744096,77.817804,3.946621
2,6.350290,190.383738,14905.393852,5.537830,333.775777,446.840605,13.983567,67.817096,4.265233
3,9.578227,205.748742,33080.588825,5.659848,356.698301,333.069911,16.984961,68.906088,3.419239
4,7.300990,182.447697,29136.338677,8.253015,333.775777,307.433303,8.730149,49.895342,4.596347


In [6]:
import numpy as np
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 0.1750640869140625  MB
******************************
Column:  ph
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Hardness
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Solids
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Chloramines
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Sulfate
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Conductivity
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Organic_carbon
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Trihalomethanes


In [7]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 0.06761932373046875  MB
******************************
Column:  ph
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Hardness
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Solids
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Chloramines
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Sulfate
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Conductivity
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Organic_carbon
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Trihalomethanes

In [ ]:
!pip install flaml

In [ ]:
y = train.pop('Potability')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
automl.fit(X_train, y_train, task="classification",metric='macro_f1',time_budget=900)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best macro_f1 on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, automl.predict(X_train)))

In [ ]:
print(classification_report(y_test, automl.predict(X_test)))

In [ ]:
y_pred = automl.predict(test)
y_pred[:5]

In [ ]:
sol = pd.DataFrame(y_pred,columns=['Potability'])
sol.head()

In [ ]:
sol.to_csv('./macro_f1.csv',index=False)